In [ ]:
import autogluon as ag
# from autogluon import TabularPrediction as task
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd ## read/write data
import re ## regex search for features name
from sklearn import preprocessing ## standardization
import math ## take log10
import numpy as np ## data handling
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
label = 'Measured logKpf or logKmw'

In [ ]:
def buildModel(dir_v,pp,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False, outputPred = False, output_file = ''):
    output_path = dir_v + output_file + '.csv'
    # load train dat
    dataPath = dir_v + "dat_training.csv"
    train_data = TabularDataset(dataPath)
    fpCols = list(filter(re.compile("^V[0-9]+$").match, list(train_data.columns)))
    drop_ref_features = list(set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set(refFeatures))
    if dir_v == "./logkpf_33Rank/":
        train_data = train_data.drop(columns=["CAS"])
    else:
        train_data = train_data.drop(columns=["CASfill0"])
    ppFeatures = list(set(list(train_data.columns)) - set(fpCols) - set(['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)']) - set([label]))
    for fp_v in fpCols:
        train_data[fp_v] = train_data[fp_v].apply(str)
    if not fp:
        train_data = train_data.drop(columns=fpCols)
    if not pp:
        train_data = train_data.drop(columns=ppFeatures)
    if len(refFeatures) != 4:
        train_data = train_data.drop(columns=drop_ref_features)
    # load test dat
    test_data = TabularDataset(dir_v + "dat_test.csv")
    for fp_v in list(filter(re.compile("^V[0-9]+$").match, list(test_data.columns))):
        test_data[fp_v] = test_data[fp_v].apply(str)
    y_test = test_data[label]  # values to predict
    test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
    # set model save_path
    save_path = dir_v + 'agModels-predictClass'  # specifies folder to store trained models
    save_path = dir_v + 'agModels-predictClass' + '-' + str(refFeatures)
    if fp:
        save_path = save_path + '-fp'
    if pp:
        save_path = save_path + '-pp'
    if auto_stack:
        save_path = save_path + '-stack'
    if hyperparameter_tune:
        save_path = save_path + '-hyperparameter_tune'
    if load_model:
        predictor = TabularPredictor.load(save_path)
    else:        
        # training
        predictor = TabularPredictor(label=label, path=save_path, eval_metric = eval_metric).fit(train_data, num_bag_folds=num_bag_folds,excluded_model_types=['NN'],
                                                                                                           auto_stack=auto_stack)
        # write training res to file
        pd.DataFrame([[dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)]],
                     columns=['dir_v','ref','padel','fp','auto_stack','hyperparameter_tune','eval_metric','num_bag_folds']).to_csv('./autoGluon2.csv',mode='a')
        train_res = predictor.leaderboard()
        train_res["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
        train_res.to_csv(output_path,mode='a')
    # write predict res to file
    if outputPred:
        print(str((dir_v,str(refFeatures))))
        train_data_nolab = train_data.drop(columns=[label])
        y_train = train_data[label]  # values to predict
        p_model_train = predictor.predict(train_data_nolab,model='WeightedEnsemble_L2')
        perf = predictor.evaluate_predictions(y_true=y_train, y_pred = p_model_train, auxiliary_metrics=True)
        
        p_model_test = predictor.predict(test_data_nolab,model='WeightedEnsemble_L2')
        df = pd.DataFrame(p_model_train)
        df["classify"] = "training"
        df_test = pd.DataFrame(p_model_test)
        df_test["classify"] = "test"
        df_bag_folds = pd.DataFrame(predictor.get_oof_pred(model= 'WeightedEnsemble_L2', transformed=False, train_data=train_data, internal_oof=False))
        df_bag_folds["classify"] = "5_bag_folds"
        df = df.append(df_test)
        df = df.append(df_bag_folds)
        df["parInfo"] = str((dir_v,str(refFeatures)))
        df.to_csv(output_path,mode='a')
    else:
        m_name_list = predictor.get_model_names()
        p_model=predictor.predict(test_data_nolab,model=m_name_list[0])
        perf = predictor.evaluate_predictions(y_true=y_test, y_pred=p_model, auxiliary_metrics=True)
        predict_df = pd.DataFrame([list(perf.values())], columns=perf.keys())
        predict_df['model'] = m_name_list[0]
        predict_df["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
        del m_name_list[0]
        if len(m_name_list) <=1:
            m_name_list = [m_name_list]
        for m_name in m_name_list:
            print(m_name)
            p_model=predictor.predict(test_data_nolab,model=m_name)
            perf = predictor.evaluate_predictions(y_true=y_test, y_pred=p_model, auxiliary_metrics=True)
            df = pd.DataFrame([list(perf.values())], columns=perf.keys())
            df['model'] = m_name
            df["parInfo"] = str((dir_v,str(refFeatures),padel,fp,auto_stack,hyperparameter_tune,eval_metric,str(num_bag_folds)))
            predict_df = predict_df.append(df)
        predict_df.to_csv(output_path,mode='a')

In [ ]:
## main

In [ ]:
dir_v_list = ["./logkpf_splitByStudy/","./logkpf_splitByRandomPercentage/","./logkpf_splitByorderedResponse/","./logkpf_splitByOrderedStructure/"]
parameter_grid = [(dir_v,padel,fp,refFeatures,auto_stack,hyperparameter_tune) for dir_v in dir_v_list for padel in [False,True] \
                for fp in [True,False] for refFeatures in [['EtOH-eq (%)(v/v)', 'Material_consolidated', 'logKow_25C','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)', 'Material_consolidated', 'Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)','Temp (C)'],
                                                          ['EtOH-eq (%)(v/v)']] \
                for auto_stack in [True,False] for hyperparameter_tune in [False]]
eval_metric = 'root_mean_squared_error'
num_bag_folds = 5
par_items = parameter_grid[0]
dir_v = par_items[0]
padel = par_items[1]
fp = par_items[2]
refFeatures = par_items[3]
auto_stack = par_items[4]
hyperparameter_tune = par_items[5]
buildModel(dir_v,padel,    fp,    refFeatures,    auto_stack,    hyperparameter_tune,    eval_metric,    num_bag_folds, load_model = False)